In [44]:
## create a tool
import requests
import json
import os
from langchain.tools import Tool

def baidu_search(query: str, num_results: int = 5) -> str:
    """
    使用 Serper API 执行百度搜索
    Args:
        query: 搜索查询字符串
        api_key: 你的 Serper API Key
        num_results: 想要返回的结果数量
    Returns:
        一个格式化的字符串，包含搜索结果摘要。
    """
    url = "https://serpapi.com/search" # Serper 的端点，但指定引擎为百度
    payload = json.dumps({
        "q": query,
        "engine": "baidu", # 指定使用百度引擎
        "num": num_results
    })
    headers = {
        'X-API-KEY': '001bd195e11581af719e2f10bfb5abd7db5f51ffb37f79b2ef7cd43af0b8e7a2',
        'Content-Type': 'application/json'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        response.raise_for_status() # 检查请求是否成功
        data = response.json()

        # 解析和组织结果
        organic_results = data.get('organic', [])
        if not organic_results:
            return "没有找到相关的搜索结果。"

        search_results = []
        for idx, result in enumerate(organic_results[:num_results], 1):
            title = result.get('title', '无标题')
            link = result.get('link', '#')
            snippet = result.get('snippet', '无摘要')
            search_results.append(f"{idx}. [{title}]({link})\n   {snippet}\n")

        return "\n".join(search_results)

    except requests.exceptions.RequestException as e:
        return f"搜索请求出错: {e}"
    except json.JSONDecodeError as e:
        return f"解析搜索结果失败: {e}"

search_tool = Tool(
    name="Baidu_Search",
    func=lambda query: baidu_search(query),
    description="使用百度搜索引擎搜索网络上的最新信息。输入应该是一个搜索查询字符串。"
)

In [45]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model(model="Qwen/Qwen3-32B", model_provider="openai", base_url="https://api.siliconflow.cn/v1", api_key="sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
tools = [search_tool]
agent_executor = create_react_agent(model, tools, checkpointer=memory)
print(agent_executor)

In [46]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================



Hello Bob! Welcome to San Francisco. How can I assist you today? If you have any questions about the local area, events, or need help with something specific, feel free to ask!


In [47]:
input_message = {
    "role": "user",
    "content": "What's the weather in beijing?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather in beijing?
================================== Ai Message ==================================
Tool Calls:
  Baidu_Search (0199377f0014a1f503b2d94f57ff7cb5)
 Call ID: 0199377f0014a1f503b2d94f57ff7cb5
  Args:
    __arg1: What's the weather in Beijing
================================= Tool Message =================================
Name: Baidu_Search

搜索请求出错: 404 Client Error: Not Found for url: https://serpapi.com/search
================================== Ai Message ==================================
Tool Calls:
  Baidu_Search (0199377f1fbec04dba8b11b3917311c5)
 Call ID: 0199377f1fbec04dba8b11b3917311c5
  Args:
    __arg1: 北京天气
================================= Tool Message =================================
Name: Baidu_Search

搜索请求出错: 404 Client Error: Not Found for url: https://serpapi.com/search
================================== Ai Message ==================================



It seems t